# This is a PushShiftAPI exploration file. 

## In here, you can pull data from PushshiftAPI and save it as CSVs

In [1]:
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import json as json
from tqdm import tqdm
api = PushshiftAPI()

## PushShiftAPI sanity test

This sanity tests scrapes the newest post on PushShiftAPI. Changes every time.

In [2]:
# gen = api.search_submissions(id = 'alusa4', filter=['url','author', 'title', 'subreddit'])
# for post in gen:
#     print(post)
# print("happened")
gen = api.search_submissions(limit=1)
for post in gen:
    print(post)

submission(all_awardings=[], allow_live_comments=False, author='byte_official', author_flair_css_class=None, author_flair_richtext=[], author_flair_text=None, author_flair_type='text', author_fullname='t2_9w7g9fi8', author_id='t2_9w7g9fi8', author_is_blocked=False, author_patreon_flair=False, author_premium=False, awarders=[], call_to_action='Learn More', can_mod_post=False, contest_mode=False, created_utc=1646175217, domain='byte.com', domain_override='byte.com', events=[], eventsOnRender=[], full_link='https://www.reddit.com/r/u_byte_official/comments/t4kt2n/its_not_too_late_to_get_the_smile_you_want_start/', gildings={}, href_url='https://www.byte.com/pages/quiz-assessment?promocode=BYTEREDDIT22&amp;utm_source=reddit&amp;utm_medium=paidsocial&amp;utm_campaign=awareness&amp;utm_content=aligners_treatment_yourself&amp;rd_ad={{AD_NAME}}&amp;rd_adgroup={{ADGROUP_NAME}}&amp;rd_adgroupid={{ADGROUP_ID}}&amp;rd_adid={{AD_ID}}&amp;rd_campaignid={{CAMPAIGN_ID}}&amp;rd_maid={{ADVERTISING_ID}}'

/Users/albertlu/opt/anaconda3/envs/Antiwork/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


## Helper function that takes in a dataset & columns and saves as CSV

In [3]:
def save_as_csv(data, cols, label):
    df = pd.DataFrame(data, columns=cols)
    df.to_csv(label + ".csv")

def merge_csv(csv1, csv2, name):
    a = pd.read_csv(csv1, index_col = 0)
    b = pd.read_csv(csv2, index_col = 0)

    merged = pd.concat([a, b], axis = 0, ignore_index = True)
    merged.to_csv(name + ".csv")
    
def print_json(post):
    json_formatted_str = json.dumps(post, indent=2)
    print(json_formatted_str)

# ID scraping function

## The function below scrapes all submission IDs & timestamps from r/antiwork from 2014-2021 and saves it as CSVs.

In [4]:
from datetime import datetime
from tqdm import tqdm
from pprint import pprint

def scrape_everything():
    all_ids = []
    params = ['id', 'created_utc']
    for year in range(2014, 2022):
        year = str(year)
        for month in range(1, 13):
            monthly_ids = []
            month = str(month)
            label = year + "-" + month
            if month == '12':
                start_epoch = int(datetime(int(year), int(month), 1).timestamp())
                end_epoch = int(datetime(int(year) + 1, 1, 1).timestamp())
            else:
                start_epoch = int(datetime(int(year), int(month), 1).timestamp())
                end_epoch = int(datetime(int(year), int(month) + 1, 1).timestamp())
            gen = api.search_submissions(after=start_epoch,
                                         before = end_epoch,
                                        subreddit='antiwork',
                                        filter=params,
                                        limit=None)
            for post in tqdm(gen):
                all_ids.append([post.id, post.created_utc])
                monthly_ids.append([post.id, post.created_utc])
            save_as_csv(monthly_ids, params, label)
            print("Got through " + year + "-" + month)
    save_as_csv(all_ids, params, "all_ids")

## This function merges two csvs

# Content scraping function

## The function below uses the submission IDs to scrape post content from r/antiwork from 2014-2021 and saves it as CSVs.

In [5]:
import pytesseract as py
import cv2
from textblob import TextBlob

py.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.0.1/bin/tesseract'

def pytesseract_sanity_check():
    img = cv2.imread(r'../data/tests/spellcheck.jpg')
    text = py.image_to_string(img)
    tb = TextBlob(text)
    autocorrected = tb.correct().strip()
    print(text)
    print(autocorrected)

def url_to_body(post_url):
    req = urlopen(post_url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1)
    text = py.image_to_string(img)
    tb = TextBlob(text)
    real_body = tb.correct()
    return real_body
pytesseract_sanity_check()

Why can't yu
spel corrctly?

 

Why can't you
spell correctly?


In [25]:
import requests
import re
from urllib.request import urlopen
import time
def scrape_content(csv_path, name):
    labels = ['title', 'body', 'is_image', 'id', 'author', 'url', 'num_comments', 'num_upvotes']
    reddit_regex = re.compile(r'https://i.redd.it/')
    ids = pd.read_csv(csv_path, index_col = 0)['id']
    prev = 0
    curr = 36900
    all_posts = []
    while(curr < 37200):
        try:
            print(curr)
            prev = curr
            curr = min(curr + 300, len(ids))
            all_ids = ','.join(ids[prev:curr])
            url = f"https://api.pushshift.io/reddit/submission/search/?ids={all_ids}"
            request = requests.get(url)
            posts = request.json()['data']
        except Exception as e:
#             curr -= 300
            print(e)
            time.sleep(180)
            continue
        for post in posts:
            try:
                body = post['selftext']
                is_image = False
                if reddit_regex.search(post['url']) != None:
                    body = url_to_body(post['url'])
                    is_image = True
                post_data = [post['title'], body, is_image, post["id"], post["author"], post["url"], post['num_comments'], post['score']]
                all_posts.append(post_data)
            except Exception as e:
#                 print(post["full_link"])
#                 print(post['url'])
                print(e)
                continue
    save_as_csv(all_posts, labels, name)
# csv_path = "../data/ids/2014/2014-9.csv"
# name = "test"
# scrape_content(csv_path, name)

def main_loop():
    for year in range(2022, 2023):
        for month in range(1, 2):
            csv_path = "../data/ids/" + str(year) + "/" + str(year) + "-"  + str(month) + ".csv"
            name = str(year) + "-" + str(month) + "1"
            scrape_content(csv_path, name)
main_loop()

36900
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found


In [24]:
def count_links(csv_path):
#     reddit_regex = re.compile(r'https://i.redd.it/' + "|" + r'imgur')
    reddit_regex = re.compile(r'https://i.redd.it/')
    imgur_regex = re.compile(r'imgur.com/')
    reddit_count = 0
    imgur_count = 0
    else_count = 0
    video_count = 0
    
    ids = pd.read_csv(csv_path, index_col = 0)['id']
    print(len(a['id']))
    prev = 0
    curr = 0
    while(curr < len(ids)):
        prev = curr
        curr = min(curr + 200, len(ids))
        all_ids = ','.join(ids[prev:curr])
        url = f"https://api.pushshift.io/reddit/submission/search/?ids={all_ids}"
        request = requests.get(url)
        posts = request.json()    
    #     print_json(post)
        data = posts['data']
        for post in data:
            try:
                body = None
                if reddit_regex.search(post['url']) != None:
                    reddit_count += 1
                elif imgur_regex.search(post['url']) != None:
                    imgur_count += 1
                elif post['selftext'] == "" or post['selftext'] == None:
                    else_count += 1
        #             print(post)
                    print(post['url'])
                    print("selftext = " + str(post['selftext']))
                if post['is_video'] == True:
                    video_count += 1    
            except:
                print("Exception")
                continue
    print("reddit count = " + str(reddit_count))
    print("imgur count = " + str(imgur_count))
    print("else count = " + str(else_count))
    print("video count = " + str(video_count))
# csv_path = "../data/2020/2020-9.csv"
# count_links(csv_path)

1469
https://youtu.be/gqUhsWdieis
selftext = 
https://v.redd.it/9q2qcq6ewop51
selftext = 
https://www.businessinsider.com/explosive-reveal-amazon-warehouse-injuries-report-2020-9
selftext = 
https://www.adeccogroup.com/futuhreinsight/5-tips-for-remote-networking/
selftext = 
https://www.reddit.com/
selftext = 
https://www.nbcsandiego.com/news/national-international/secret-man-cave-discovered-below-grand-central-platform-3-mta-employees-busted/2411770/
selftext = 
/r/legaladvice/comments/j2soq7/my_job_is_making_everyone_sign_a_constitutional/
selftext = 
https://jacobinmag.com/2020/09/overwork-overtime-workfare-unions-socialism
selftext = 
/r/TrueOffMyChest/comments/j2qily/i_grew_up_poor_as_hell_and_managed_to_bootstrap/
selftext = 
https://www.reddit.com/gallery/j0tawu
selftext = 
https://lifehacker.com/how-to-make-it-look-like-youre-working-from-home-when-y-1842273907
selftext = 
https://youtu.be/qGtT2EWtdW4
selftext = 
https://v.redd.it/0prhkew2ipp51
selftext = 
https://www.goodnewsn

https://www.wfh-tracker.com
selftext = 
https://allthatsinteresting.com/woman-boiled-molten-caramel?fbclid=IwAR0DAqPVXAp_a8Eb8asda7XTkhx21DtnXIrS6DueftST_rfTGo9IIzgAxNU
selftext = 
https://www.ndtv.com/world-news/jpmorgan-says-work-from-home-output-down-mondays-and-fridays-worst-2295946
selftext = 
https://www.fastcompany.com/90550015/we-were-shocked-rand-study-uncovers-massive-income-shift-to-the-top-1
selftext = 
https://www.abc.net.au/news/2020-09-15/backpacker-farm-workers-speak-of-wage-exploitation/12545294?nw=0
selftext = 
/r/Anarchy101/comments/itp9d4/whats_the_point_of_progress_if_most_are_worse_off/
selftext = 
https://towardsdatascience.com/how-this-a-i-became-a-communist-ddf9146bc147
selftext = 
http://misc-stuff.terraaeon.com/articles/engineering-deadlines.html
selftext = 
https://www.reddit.com/r/ABoringDystopia/comments/isla7f/man_forced_to_work_even_after_life_changing/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf
selftext = 
https://youtu.be/EO_lcDZ5mzs
s

https://www.youtube.com/watch?v=bPrDV_5qrRE
selftext = 
/r/LateStageCapitalism/comments/iljr2s/australias_biggest_telco_urges_staff_to_work_for/
selftext = 
/r/confessions/comments/ilvjk9/i_have_worked_16_hours_a_week_on_a_85k_salary_for/
selftext = 
https://v.redd.it/7h2tipvvjuk51
selftext = 
https://salsamoment.wixsite.com/website/post/you-are-not-immune-to-propaganda
selftext = 
https://thebaffler.com/salvos/whats-the-point-if-we-cant-have-fun
selftext = 
https://twitter.com/helloalegria/status/1301290315593191426
selftext = 
https://www.theguardian.com/books/2020/sep/03/david-graeber-anthropologist-and-author-of-bullshit-jobs-dies-aged-59
selftext = 
/r/quotes/comments/imaw4n/i_am_somehow_less_interested_in_the_weight_and/
selftext = 
https://www.reddit.com/r/antiwork/comments/imoueg/before_having_more_political_knowledge_has_anyone/
selftext = 
https://www.youtube.com/watch?v=z6cvmnKTJLA
selftext = 
https://www.adeccogroup.com/futuhreinsight/debunking-the-myths-of-generation-z-as-

In [26]:
merge_csv("2022-1.csv", "2022-11.csv", "2022-1")